# imports

In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, GlobalAveragePooling2D, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.applications import Xception, DenseNet201
from tensorflow.keras.callbacks import LearningRateScheduler
import random
from PIL import Image
import shutil

# global variables

In [25]:
# image size options: 192, 224, 311, 512
IMAGE_DIMENSION = 192
VECTOR_LEN = IMAGE_DIMENSION**2
NUM_CLASS = 96

train_dir = f'data/jpeg-{IMAGE_DIMENSION}x{IMAGE_DIMENSION}/train'
val_dir = f'data/jpeg-{IMAGE_DIMENSION}x{IMAGE_DIMENSION}/val'
test_dir = f'data/jpeg-{IMAGE_DIMENSION}x{IMAGE_DIMENSION}/test'

BATCH_SIZE = 10
TRAIN_BATCH_SIZE = BATCH_SIZE
VAL_BATCH_SIZE = BATCH_SIZE
TEST_BATCH_SIZE = BATCH_SIZE

# eda

# generate datasets

In [3]:
train_generator = ImageDataGenerator(rescale=1./255).flow_from_directory(
    train_dir,
    target_size=(IMAGE_DIMENSION, IMAGE_DIMENSION), 
    batch_size=TRAIN_BATCH_SIZE,
    shuffle=True 
)

val_generator = ImageDataGenerator().flow_from_directory(
    val_dir,
    target_size=(IMAGE_DIMENSION, IMAGE_DIMENSION),
    batch_size=VAL_BATCH_SIZE,
    shuffle=True
)

test_generator = ImageDataGenerator().flow_from_directory(
    test_dir,
    target_size=(IMAGE_DIMENSION, IMAGE_DIMENSION),
    batch_size=TEST_BATCH_SIZE,
    shuffle=False
)

Found 11459 images belonging to 104 classes.
Found 3710 images belonging to 104 classes.
Found 1294 images belonging to 104 classes.


### defining functions

In [4]:
def get_subfolder_list(source_folder):
    # returns a list of all the subfolders in the source_folder
    class_list_dir = []

    for file in os.listdir(source_folder):
        d = os.path.join(source_folder, file)
        if os.path.isdir(d):
            class_list_dir.append(d)

    return class_list_dir

def map_classes(subfolder_list):
    # returns a dict with the flower as the key, and (count, directory) as the values
    class_dict = {}

    for class_folder in subfolder_list:
        file_count = sum(len(files) for _, _, files in os.walk(class_folder))
        class_dict[class_folder[24:]] = file_count, class_folder
        
    return class_dict

def get_flower_count(class_dict):
    # returns a list of the number of flowers found in the dict
    flower_count = []

    for flower in list(class_dict.values()):
        flower_count.append(flower[0])

    return flower_count

In [5]:
def get_metrics(flower_count):
    # returns basic metrics when given a list of flower value count
    class_std = np.std(flower_count)
    class_max = max(flower_count)
    class_min = min(flower_count)
    class_mean = np.mean(flower_count)
    class_first_quartile = np.percentile(flower_count, 25)
    class_third_quartile = np.percentile(flower_count, 75)
    class_tenth_percentile = np.percentile(flower_count, 10)
    class_fifth_percentile = np.percentile(flower_count, 5)

    print(f'0. standard deviation: {class_std}')
    print(f'1. max: {class_max}')
    print(f'2. min: {class_min}')
    print(f'3. mean: {class_mean}')
    print(f'4. 25%: {class_first_quartile}')
    print(f'5. 75%: {class_third_quartile}')
    print(f'6. 10%: {class_tenth_percentile}')
    print(f'7. 5%: {class_fifth_percentile}')
    
    return (class_std, class_max, class_min, class_mean, class_first_quartile,
class_third_quartile, class_tenth_percentile, class_fifth_percentile)

### testing above functions / eda

In [6]:
train_subfolders = get_subfolder_list(train_dir)
train_dict = map_classes(train_subfolders)
train_flower_count = get_flower_count(train_dict)

In [7]:
train_subfolders[:5]

['data/jpeg-192x192/train/toad lily',
 'data/jpeg-192x192/train/love in the mist',
 'data/jpeg-192x192/train/monkshood',
 'data/jpeg-192x192/train/azalea',
 'data/jpeg-192x192/train/fritillary']

In [8]:
get_metrics(train_flower_count)

0. standard deviation: 132.99130714962862
1. max: 707
2. min: 16
3. mean: 111.1826923076923
4. 25%: 30.5
5. 75%: 113.5
6. 10%: 19.0
7. 5%: 17.15


(132.99130714962862, 707, 16, 111.1826923076923, 30.5, 113.5, 19.0, 17.15)

In [9]:
val_subfolders = get_subfolder_list(val_dir)
val_dict = map_classes(val_subfolders)
val_flower_count = get_flower_count(val_dict)

In [10]:
get_metrics(val_flower_count)

0. standard deviation: 42.990624407913046
1. max: 228
2. min: 5
3. mean: 35.69230769230769
4. 25%: 9.0
5. 75%: 37.0
6. 10%: 6.0
7. 5%: 6.0


(42.990624407913046, 228, 5, 35.69230769230769, 9.0, 37.0, 6.0, 6.0)

# preprocessing

### more functions

In [17]:
def copy_subfolder(source):
    # copies source folder to a new directory with '_new' attached to the end of the folder name
    prev_dir_index = source.rfind('/')
    destination = source[:prev_dir_index] + '_new' + source[prev_dir_index:]
    
    
    if not os.path.exists(destination):
        result = shutil.copytree(source, destination, symlinks=False, ignore=None, 
                                 copy_function=shutil.copy2, ignore_dangling_symlinks=False, 
                                 dirs_exist_ok=False)
    else:
        print(f'{destination} already exists')
        
    return result
    
def item_count(folder):
    # helper function to identify which folders to remove
    file_count = sum(len(files) for _, _, files in os.walk(folder))
    
    return file_count
    
def get_short_list(subfolder_list, n):
    # find a list of classes that are divided by the specified n value
    temp_dict = map_classes(subfolder_list)
    temp_flower_count = get_flower_count(temp_dict)
    
    move_list = []
    ignore_list = []
    
    percent_cutoff = np.percentile(temp_flower_count, n)
    
    for subfolder in subfolder_list:
        if item_count(subfolder) >= percent_cutoff:
            move_list.append(subfolder)
        else:
            ignore_list.append(subfolder)
            
    return move_list, ignore_list
    
def trim(subfolder_list):
    # main function used to copy classes into new train, val, test 
    for subfolder in subfolder_list:
        copy_subfolder(subfolder)

        val_subfolder = subfolder.replace('/train/', '/val/')
        copy_subfolder(val_subfolder)

        test_subfolder = subfolder.replace('/train/', '/test/')
        copy_subfolder(test_subfolder)

In [12]:
move_list, ignore_list = get_short_list(train_subfolders, 10)

In [13]:
move_list[:5]

['data/jpeg-192x192/train/toad lily',
 'data/jpeg-192x192/train/love in the mist',
 'data/jpeg-192x192/train/monkshood',
 'data/jpeg-192x192/train/azalea',
 'data/jpeg-192x192/train/fritillary']

In [ ]:
trim(move_list)

In [64]:
src = 'data/jpeg-192x192/train/iris'

copy_subfolder(src)

data/jpeg-192x192/train_new/iris
23


FileExistsError: [Errno 17] File exists: 'data/jpeg-192x192/train_new/iris'

In [29]:
# updating the directories as new folders are made 
train_dir = f'data/jpeg-{IMAGE_DIMENSION}x{IMAGE_DIMENSION}/train_new'
val_dir = f'data/jpeg-{IMAGE_DIMENSION}x{IMAGE_DIMENSION}/val_new'
test_dir = f'data/jpeg-{IMAGE_DIMENSION}x{IMAGE_DIMENSION}/test_new'

# generate datasets

In [19]:
train_generator = ImageDataGenerator(rescale=1./255).flow_from_directory(
    train_dir,
    target_size=(IMAGE_DIMENSION, IMAGE_DIMENSION), 
    batch_size=TRAIN_BATCH_SIZE,
    shuffle=True 
)

val_generator = ImageDataGenerator().flow_from_directory(
    val_dir,
    target_size=(IMAGE_DIMENSION, IMAGE_DIMENSION),
    batch_size=VAL_BATCH_SIZE,
    shuffle=True
)

test_generator = ImageDataGenerator().flow_from_directory(
    test_dir,
    target_size=(IMAGE_DIMENSION, IMAGE_DIMENSION),
    batch_size=TEST_BATCH_SIZE,
    shuffle=False
)

Found 11331 images belonging to 96 classes.
Found 3666 images belonging to 96 classes.
Found 1271 images belonging to 96 classes.


# modeling

In [20]:
def scheduler(epoch, lr):
    if epoch < 10:
        return lr
    else:
        return lr * tf.math.exp(-0.1)
    
lr_callback = LearningRateScheduler(scheduler)

In [32]:
train_generator = ImageDataGenerator(rescale=1./255, 
                                     horizontal_flip=True, 
                                     rotation_range=45,
                                     vertical_flip=False,
                                     brightness_range=[0.75,1.25],
                                     zoom_range=0.2,
                                     shear_range=0.2
                                    ).flow_from_directory(
    train_dir,
    target_size=(IMAGE_DIMENSION, IMAGE_DIMENSION), 
    batch_size=TRAIN_BATCH_SIZE,
    shuffle=True 
)

val_generator = ImageDataGenerator().flow_from_directory(
    val_dir,
    target_size=(IMAGE_DIMENSION, IMAGE_DIMENSION),
    batch_size=VAL_BATCH_SIZE,
    shuffle=True
)

test_generator = ImageDataGenerator().flow_from_directory(
    test_dir,
    target_size=(IMAGE_DIMENSION, IMAGE_DIMENSION),
    batch_size=TEST_BATCH_SIZE,
    shuffle=False
)

Found 11331 images belonging to 96 classes.
Found 3666 images belonging to 96 classes.
Found 1271 images belonging to 96 classes.


In [33]:
model = models.Sequential()
model.add(layers.Conv2D(filters=32, 
                        kernel_size=(2,2),
                        strides=(1,1),
                        activation='relu',
                        padding = 'same',
                        input_shape=(IMAGE_DIMENSION, IMAGE_DIMENSION, 3),
                        data_format = 'channels_last'))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(64, (3,3), activation='relu'))
model.add(layers.MaxPooling2D((3,4)))
model.add(layers.Flatten())
model.add(layers.Dropout(0.5, input_shape=(IMAGE_DIMENSION, IMAGE_DIMENSION, 3)))
model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dense(NUM_CLASS)) # output layer
model.add(layers.Activation('sigmoid'))

model.summary()

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 192, 192, 32)      416       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 96, 96, 32)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 94, 94, 64)        18496     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 31, 23, 64)        0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 45632)             0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 45632)             0         
_________________________________________________________________
dense_4 (Dense)              (None, 32)               

In [34]:
model.fit(train_generator, steps_per_epoch=11331 // BATCH_SIZE, epochs=20,
          validation_data=val_generator, callbacks=lr_callback, use_multiprocessing=False)

Epoch 1/20
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: unsupported operand type(s) for -: 'NoneType' and 'int'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: unsupported operand type(s) for -: 'NoneType' and 'int'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
1133/1133 [==============================] - ETA: 0s - loss: 4.1504 - accuracy: 0.0831WARNING:tensorflow:AutoGraph could not transform <function Model.make_test_function.<locals>.test_function at 0x2aa828430> and will run it as-is.
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOS

### simple model

In [35]:
model = models.Sequential()
model.add(layers.Conv2D(filters=64, 
                        kernel_size=(2,2),
                        activation='relu',
                        padding = 'same',
                        input_shape=(IMAGE_DIMENSION, IMAGE_DIMENSION, 3),
                        data_format = 'channels_last'))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Flatten())
model.add(layers.Dense(NUM_CLASS)) # output layer
model.add(layers.Activation('sigmoid'))

model.summary()

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# training begins
model.fit(train_generator, steps_per_epoch=11331 // BATCH_SIZE, epochs=20,
          validation_data=val_generator, callbacks=lr_callback, use_multiprocessing=False)

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 192, 192, 64)      832       
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 96, 96, 64)        0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 589824)            0         
_________________________________________________________________
dense_6 (Dense)              (None, 96)                56623200  
_________________________________________________________________
activation_3 (Activation)    (None, 96)                0         
Total params: 56,624,032
Trainable params: 56,624,032
Non-trainable params: 0
_________________________________________________________________
Epoch 1/20
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux

KeyboardInterrupt: 

### simple model 1

In [36]:
model = models.Sequential()
model.add(layers.Conv2D(filters=32, 
                        kernel_size=(2,2),
                        activation='relu',
                        padding = 'same',
                        input_shape=(IMAGE_DIMENSION, IMAGE_DIMENSION, 3),
                        data_format = 'channels_last'))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Flatten())
model.add(layers.Dense(NUM_CLASS)) # output layer
model.add(layers.Activation('sigmoid'))

model.summary()

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# training begins
model.fit(train_generator, steps_per_epoch=11331 // BATCH_SIZE, epochs=20,
          validation_data=val_generator, callbacks=lr_callback, use_multiprocessing=False)

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_7 (Conv2D)            (None, 192, 192, 32)      416       
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 96, 96, 32)        0         
_________________________________________________________________
flatten_4 (Flatten)          (None, 294912)            0         
_________________________________________________________________
dense_7 (Dense)              (None, 96)                28311648  
_________________________________________________________________
activation_4 (Activation)    (None, 96)                0         
Total params: 28,312,064
Trainable params: 28,312,064
Non-trainable params: 0
_________________________________________________________________
Epoch 1/20
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux

KeyboardInterrupt: 

### simple 2 - another layer

In [37]:
model = models.Sequential()
model.add(layers.Conv2D(filters=32, 
                        kernel_size=(2,2),
                        activation='relu',
                        padding = 'same',
                        input_shape=(IMAGE_DIMENSION, IMAGE_DIMENSION, 3),
                        data_format = 'channels_last'))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(16, (2,2), activation='relu'))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Flatten())
model.add(layers.Dense(NUM_CLASS)) # output layer
model.add(layers.Activation('sigmoid'))

model.summary()

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# training begins
model.fit(train_generator, steps_per_epoch=11331 // BATCH_SIZE, epochs=20,
          validation_data=val_generator, callbacks=lr_callback, use_multiprocessing=False)

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_8 (Conv2D)            (None, 192, 192, 32)      416       
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 96, 96, 32)        0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 95, 95, 16)        2064      
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 47, 47, 16)        0         
_________________________________________________________________
flatten_5 (Flatten)          (None, 35344)             0         
_________________________________________________________________
dense_8 (Dense)              (None, 96)                3393120   
_________________________________________________________________
activation_5 (Activation)    (None, 96)               

### simple 3 - change the kernel_size

In [38]:
model = models.Sequential()
model.add(layers.Conv2D(filters=32, 
                        kernel_size=(2,2),
                        activation='relu',
                        padding = 'same',
                        input_shape=(IMAGE_DIMENSION, IMAGE_DIMENSION, 3),
                        data_format = 'channels_last'))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(16, (3,3), activation='relu'))
model.add(layers.MaxPooling2D((3,3)))
model.add(layers.Flatten())
model.add(layers.Dense(NUM_CLASS)) # output layer
model.add(layers.Activation('sigmoid'))

model.summary()

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# training begins
model.fit(train_generator, steps_per_epoch=11331 // BATCH_SIZE, epochs=20,
          validation_data=val_generator, callbacks=lr_callback, use_multiprocessing=False)

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_10 (Conv2D)           (None, 192, 192, 32)      416       
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 96, 96, 32)        0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 94, 94, 16)        4624      
_________________________________________________________________
max_pooling2d_11 (MaxPooling (None, 31, 31, 16)        0         
_________________________________________________________________
flatten_6 (Flatten)          (None, 15376)             0         
_________________________________________________________________
dense_9 (Dense)              (None, 96)                1476192   
_________________________________________________________________
activation_6 (Activation)    (None, 96)               

### simple 4 - batch size increase

In [39]:
BATCH_SIZE = 64
TRAIN_BATCH_SIZE = BATCH_SIZE
VAL_BATCH_SIZE = BATCH_SIZE
TEST_BATCH_SIZE = BATCH_SIZE

In [40]:
train_generator = ImageDataGenerator(rescale=1./255, 
                                     horizontal_flip=True, 
                                     rotation_range=45,
                                     vertical_flip=False,
                                     brightness_range=[0.75,1.25],
                                     zoom_range=0.2,
                                     shear_range=0.2
                                    ).flow_from_directory(
    train_dir,
    target_size=(IMAGE_DIMENSION, IMAGE_DIMENSION), 
    batch_size=TRAIN_BATCH_SIZE,
    shuffle=True 
)

val_generator = ImageDataGenerator().flow_from_directory(
    val_dir,
    target_size=(IMAGE_DIMENSION, IMAGE_DIMENSION),
    batch_size=VAL_BATCH_SIZE,
    shuffle=True
)

test_generator = ImageDataGenerator().flow_from_directory(
    test_dir,
    target_size=(IMAGE_DIMENSION, IMAGE_DIMENSION),
    batch_size=TEST_BATCH_SIZE,
    shuffle=False
)

Found 11331 images belonging to 96 classes.
Found 3666 images belonging to 96 classes.
Found 1271 images belonging to 96 classes.


In [41]:
model = models.Sequential()
model.add(layers.Conv2D(filters=32, 
                        kernel_size=(2,2),
                        activation='relu',
                        padding = 'same',
                        input_shape=(IMAGE_DIMENSION, IMAGE_DIMENSION, 3),
                        data_format = 'channels_last'))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(16, (3,3), activation='relu'))
model.add(layers.MaxPooling2D((3,3)))
model.add(layers.Flatten())
model.add(layers.Dense(NUM_CLASS)) # output layer
model.add(layers.Activation('sigmoid'))

model.summary()

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# training begins
model.fit(train_generator, steps_per_epoch=11331 // BATCH_SIZE, epochs=20,
          validation_data=val_generator, callbacks=lr_callback, use_multiprocessing=False)

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_12 (Conv2D)           (None, 192, 192, 32)      416       
_________________________________________________________________
max_pooling2d_12 (MaxPooling (None, 96, 96, 32)        0         
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 94, 94, 16)        4624      
_________________________________________________________________
max_pooling2d_13 (MaxPooling (None, 31, 31, 16)        0         
_________________________________________________________________
flatten_7 (Flatten)          (None, 15376)             0         
_________________________________________________________________
dense_10 (Dense)             (None, 96)                1476192   
_________________________________________________________________
activation_7 (Activation)    (None, 96)               

### simple 5 - further batch size increase

In [42]:
BATCH_SIZE = 128
TRAIN_BATCH_SIZE = BATCH_SIZE
VAL_BATCH_SIZE = BATCH_SIZE
TEST_BATCH_SIZE = BATCH_SIZE

In [43]:
train_generator = ImageDataGenerator(rescale=1./255, 
                                     horizontal_flip=True, 
                                     rotation_range=45,
                                     vertical_flip=False,
                                     brightness_range=[0.75,1.25],
                                     zoom_range=0.2,
                                     shear_range=0.2
                                    ).flow_from_directory(
    train_dir,
    target_size=(IMAGE_DIMENSION, IMAGE_DIMENSION), 
    batch_size=TRAIN_BATCH_SIZE,
    shuffle=True 
)

val_generator = ImageDataGenerator().flow_from_directory(
    val_dir,
    target_size=(IMAGE_DIMENSION, IMAGE_DIMENSION),
    batch_size=VAL_BATCH_SIZE,
    shuffle=True
)

test_generator = ImageDataGenerator().flow_from_directory(
    test_dir,
    target_size=(IMAGE_DIMENSION, IMAGE_DIMENSION),
    batch_size=TEST_BATCH_SIZE,
    shuffle=False
)

Found 11331 images belonging to 96 classes.
Found 3666 images belonging to 96 classes.
Found 1271 images belonging to 96 classes.


In [44]:
model = models.Sequential()
model.add(layers.Conv2D(filters=32, 
                        kernel_size=(2,2),
                        activation='relu',
                        padding = 'same',
                        input_shape=(IMAGE_DIMENSION, IMAGE_DIMENSION, 3),
                        data_format = 'channels_last'))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(16, (3,3), activation='relu'))
model.add(layers.MaxPooling2D((3,3)))
model.add(layers.Flatten())
model.add(layers.Dense(NUM_CLASS)) # output layer
model.add(layers.Activation('sigmoid'))

model.summary()

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# training begins
model.fit(train_generator, steps_per_epoch=11331 // BATCH_SIZE, epochs=20,
          validation_data=val_generator, callbacks=lr_callback, use_multiprocessing=False)

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_14 (Conv2D)           (None, 192, 192, 32)      416       
_________________________________________________________________
max_pooling2d_14 (MaxPooling (None, 96, 96, 32)        0         
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 94, 94, 16)        4624      
_________________________________________________________________
max_pooling2d_15 (MaxPooling (None, 31, 31, 16)        0         
_________________________________________________________________
flatten_8 (Flatten)          (None, 15376)             0         
_________________________________________________________________
dense_11 (Dense)             (None, 96)                1476192   
_________________________________________________________________
activation_8 (Activation)    (None, 96)               

### simple 6 - revert to simple 4. add another dense layer

In [45]:
BATCH_SIZE = 64
TRAIN_BATCH_SIZE = BATCH_SIZE
VAL_BATCH_SIZE = BATCH_SIZE
TEST_BATCH_SIZE = BATCH_SIZE

In [46]:
train_generator = ImageDataGenerator(rescale=1./255, 
                                     horizontal_flip=True, 
                                     rotation_range=45,
                                     vertical_flip=False,
                                     brightness_range=[0.75,1.25],
                                     zoom_range=0.2,
                                     shear_range=0.2
                                    ).flow_from_directory(
    train_dir,
    target_size=(IMAGE_DIMENSION, IMAGE_DIMENSION), 
    batch_size=TRAIN_BATCH_SIZE,
    shuffle=True 
)

val_generator = ImageDataGenerator().flow_from_directory(
    val_dir,
    target_size=(IMAGE_DIMENSION, IMAGE_DIMENSION),
    batch_size=VAL_BATCH_SIZE,
    shuffle=True
)

test_generator = ImageDataGenerator().flow_from_directory(
    test_dir,
    target_size=(IMAGE_DIMENSION, IMAGE_DIMENSION),
    batch_size=TEST_BATCH_SIZE,
    shuffle=False
)

Found 11331 images belonging to 96 classes.
Found 3666 images belonging to 96 classes.
Found 1271 images belonging to 96 classes.


In [47]:
model = models.Sequential()
model.add(layers.Conv2D(filters=32, 
                        kernel_size=(2,2),
                        activation='relu',
                        padding = 'same',
                        input_shape=(IMAGE_DIMENSION, IMAGE_DIMENSION, 3),
                        data_format = 'channels_last'))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(16, (3,3), activation='relu'))
model.add(layers.MaxPooling2D((3,3)))
model.add(layers.Flatten())
model.add(layers.Dense(64))
model.add(layers.Dense(NUM_CLASS)) # output layer
model.add(layers.Activation('sigmoid'))

model.summary()

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# training begins
model.fit(train_generator, steps_per_epoch=11331 // BATCH_SIZE, epochs=20,
          validation_data=val_generator, callbacks=lr_callback, use_multiprocessing=False)

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_16 (Conv2D)           (None, 192, 192, 32)      416       
_________________________________________________________________
max_pooling2d_16 (MaxPooling (None, 96, 96, 32)        0         
_________________________________________________________________
conv2d_17 (Conv2D)           (None, 94, 94, 16)        4624      
_________________________________________________________________
max_pooling2d_17 (MaxPooling (None, 31, 31, 16)        0         
_________________________________________________________________
flatten_9 (Flatten)          (None, 15376)             0         
_________________________________________________________________
dense_12 (Dense)             (None, 64)                984128    
_________________________________________________________________
dense_13 (Dense)             (None, 96)               

### simple 7 - adjust learning rate

In [51]:
def scheduler(epoch, lr):
    if epoch < 10:
        return lr
    else:
        return 0.00001
    
lr_callback = LearningRateScheduler(scheduler)

In [52]:
BATCH_SIZE = 64
TRAIN_BATCH_SIZE = BATCH_SIZE
VAL_BATCH_SIZE = BATCH_SIZE
TEST_BATCH_SIZE = BATCH_SIZE

In [53]:
train_generator = ImageDataGenerator(rescale=1./255, 
                                     horizontal_flip=True, 
                                     rotation_range=45,
                                     vertical_flip=False,
                                     brightness_range=[0.75,1.25],
                                     zoom_range=0.2,
                                     shear_range=0.2
                                    ).flow_from_directory(
    train_dir,
    target_size=(IMAGE_DIMENSION, IMAGE_DIMENSION), 
    batch_size=TRAIN_BATCH_SIZE,
    shuffle=True 
)

val_generator = ImageDataGenerator().flow_from_directory(
    val_dir,
    target_size=(IMAGE_DIMENSION, IMAGE_DIMENSION),
    batch_size=VAL_BATCH_SIZE,
    shuffle=True
)

test_generator = ImageDataGenerator().flow_from_directory(
    test_dir,
    target_size=(IMAGE_DIMENSION, IMAGE_DIMENSION),
    batch_size=TEST_BATCH_SIZE,
    shuffle=False
)

Found 11331 images belonging to 96 classes.
Found 3666 images belonging to 96 classes.
Found 1271 images belonging to 96 classes.


In [55]:
model = models.Sequential()
model.add(layers.Conv2D(filters=32, 
                        kernel_size=(2,2),
                        activation='relu',
                        padding = 'same',
                        input_shape=(IMAGE_DIMENSION, IMAGE_DIMENSION, 3),
                        data_format = 'channels_last'))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(16, (3,3), activation='relu'))
model.add(layers.MaxPooling2D((3,3)))
model.add(layers.Flatten())
model.add(layers.Dense(64))
model.add(layers.Dense(NUM_CLASS)) # output layer
model.add(layers.Activation('sigmoid'))

model.summary()

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# training begins
model.fit(train_generator, steps_per_epoch=11331 // BATCH_SIZE, epochs=20,
          validation_data=val_generator, callbacks=lr_callback, use_multiprocessing=False)

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_20 (Conv2D)           (None, 192, 192, 32)      416       
_________________________________________________________________
max_pooling2d_20 (MaxPooling (None, 96, 96, 32)        0         
_________________________________________________________________
conv2d_21 (Conv2D)           (None, 94, 94, 16)        4624      
_________________________________________________________________
max_pooling2d_21 (MaxPooling (None, 31, 31, 16)        0         
_________________________________________________________________
flatten_11 (Flatten)         (None, 15376)             0         
_________________________________________________________________
dense_16 (Dense)             (None, 64)                984128    
_________________________________________________________________
dense_17 (Dense)             (None, 96)              

### simple 8 - improving generalization. add dropout layer

In [51]:
def scheduler(epoch, lr):
    if epoch < 10:
        return lr
    else:
        return 0.00001
    
lr_callback = LearningRateScheduler(scheduler)

In [52]:
BATCH_SIZE = 64
TRAIN_BATCH_SIZE = BATCH_SIZE
VAL_BATCH_SIZE = BATCH_SIZE
TEST_BATCH_SIZE = BATCH_SIZE

In [53]:
train_generator = ImageDataGenerator(rescale=1./255, 
                                     horizontal_flip=True, 
                                     rotation_range=45,
                                     vertical_flip=False,
                                     brightness_range=[0.75,1.25],
                                     zoom_range=0.2,
                                     shear_range=0.2
                                    ).flow_from_directory(
    train_dir,
    target_size=(IMAGE_DIMENSION, IMAGE_DIMENSION), 
    batch_size=TRAIN_BATCH_SIZE,
    shuffle=True 
)

val_generator = ImageDataGenerator().flow_from_directory(
    val_dir,
    target_size=(IMAGE_DIMENSION, IMAGE_DIMENSION),
    batch_size=VAL_BATCH_SIZE,
    shuffle=True
)

test_generator = ImageDataGenerator().flow_from_directory(
    test_dir,
    target_size=(IMAGE_DIMENSION, IMAGE_DIMENSION),
    batch_size=TEST_BATCH_SIZE,
    shuffle=False
)

Found 11331 images belonging to 96 classes.
Found 3666 images belonging to 96 classes.
Found 1271 images belonging to 96 classes.


In [56]:
model = models.Sequential()
model.add(layers.Conv2D(filters=32, 
                        kernel_size=(2,2),
                        activation='relu',
                        padding = 'same',
                        input_shape=(IMAGE_DIMENSION, IMAGE_DIMENSION, 3),
                        data_format = 'channels_last'))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(16, (3,3), activation='relu'))
model.add(layers.MaxPooling2D((3,3)))
model.add(layers.Flatten())
model.add(layers.Dropout(0.3))
model.add(layers.Dense(64))
model.add(layers.Dense(NUM_CLASS)) # output layer
model.add(layers.Activation('sigmoid'))

model.summary()

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# training begins
model.fit(train_generator, steps_per_epoch=11331 // BATCH_SIZE, epochs=20,
          validation_data=val_generator, callbacks=lr_callback, use_multiprocessing=False)

Model: "sequential_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_22 (Conv2D)           (None, 192, 192, 32)      416       
_________________________________________________________________
max_pooling2d_22 (MaxPooling (None, 96, 96, 32)        0         
_________________________________________________________________
conv2d_23 (Conv2D)           (None, 94, 94, 16)        4624      
_________________________________________________________________
max_pooling2d_23 (MaxPooling (None, 31, 31, 16)        0         
_________________________________________________________________
flatten_12 (Flatten)         (None, 15376)             0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 15376)             0         
_________________________________________________________________
dense_18 (Dense)             (None, 64)              

### simple 9 - more generalization. increase dropout %

In [57]:
def scheduler(epoch, lr):
    if epoch < 10:
        return lr
    else:
        return 0.00001
    
lr_callback = LearningRateScheduler(scheduler)

In [58]:
BATCH_SIZE = 64
TRAIN_BATCH_SIZE = BATCH_SIZE
VAL_BATCH_SIZE = BATCH_SIZE
TEST_BATCH_SIZE = BATCH_SIZE

In [59]:
train_generator = ImageDataGenerator(rescale=1./255, 
                                     horizontal_flip=True, 
                                     rotation_range=45,
                                     vertical_flip=False,
                                     brightness_range=[0.75,1.25],
                                     zoom_range=0.2,
                                     shear_range=0.2
                                    ).flow_from_directory(
    train_dir,
    target_size=(IMAGE_DIMENSION, IMAGE_DIMENSION), 
    batch_size=TRAIN_BATCH_SIZE,
    shuffle=True 
)

val_generator = ImageDataGenerator().flow_from_directory(
    val_dir,
    target_size=(IMAGE_DIMENSION, IMAGE_DIMENSION),
    batch_size=VAL_BATCH_SIZE,
    shuffle=True
)

test_generator = ImageDataGenerator().flow_from_directory(
    test_dir,
    target_size=(IMAGE_DIMENSION, IMAGE_DIMENSION),
    batch_size=TEST_BATCH_SIZE,
    shuffle=False
)

Found 11331 images belonging to 96 classes.
Found 3666 images belonging to 96 classes.
Found 1271 images belonging to 96 classes.


In [60]:
model = models.Sequential()
model.add(layers.Conv2D(filters=32, 
                        kernel_size=(2,2),
                        activation='relu',
                        padding = 'same',
                        input_shape=(IMAGE_DIMENSION, IMAGE_DIMENSION, 3),
                        data_format = 'channels_last'))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(16, (3,3), activation='relu'))
model.add(layers.MaxPooling2D((3,3)))
model.add(layers.Flatten())
model.add(layers.Dropoutout(0.6))
model.add(layers.Dense(64))
model.add(layers.Dense(NUM_CLASS)) # output layer
model.add(layers.Activation('sigmoid'))

model.summary()

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# training begins
model.fit(train_generator, steps_per_epoch=11331 // BATCH_SIZE, epochs=20,
          validation_data=val_generator, callbacks=lr_callback, use_multiprocessing=False)

Model: "sequential_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_24 (Conv2D)           (None, 192, 192, 32)      416       
_________________________________________________________________
max_pooling2d_24 (MaxPooling (None, 96, 96, 32)        0         
_________________________________________________________________
conv2d_25 (Conv2D)           (None, 94, 94, 16)        4624      
_________________________________________________________________
max_pooling2d_25 (MaxPooling (None, 31, 31, 16)        0         
_________________________________________________________________
flatten_13 (Flatten)         (None, 15376)             0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 15376)             0         
_________________________________________________________________
dense_20 (Dense)             (None, 64)              

### simple 10. revert to simple 7 and apply normalization

In [61]:
def scheduler(epoch, lr):
    if epoch < 10:
        return lr
    else:
        return 0.00001
    
lr_callback = LearningRateScheduler(scheduler)

In [62]:
BATCH_SIZE = 64
TRAIN_BATCH_SIZE = BATCH_SIZE
VAL_BATCH_SIZE = BATCH_SIZE
TEST_BATCH_SIZE = BATCH_SIZE

In [63]:
train_generator = ImageDataGenerator(rescale=1./255, 
                                     horizontal_flip=True, 
                                     rotation_range=45,
                                     vertical_flip=False,
                                     brightness_range=[0.75,1.25],
                                     zoom_range=0.2,
                                     shear_range=0.2
                                    ).flow_from_directory(
    train_dir,
    target_size=(IMAGE_DIMENSION, IMAGE_DIMENSION), 
    batch_size=TRAIN_BATCH_SIZE,
    shuffle=True 
)

val_generator = ImageDataGenerator().flow_from_directory(
    val_dir,
    target_size=(IMAGE_DIMENSION, IMAGE_DIMENSION),
    batch_size=VAL_BATCH_SIZE,
    shuffle=True
)

test_generator = ImageDataGenerator().flow_from_directory(
    test_dir,
    target_size=(IMAGE_DIMENSION, IMAGE_DIMENSION),
    batch_size=TEST_BATCH_SIZE,
    shuffle=False
)

Found 11331 images belonging to 96 classes.
Found 3666 images belonging to 96 classes.
Found 1271 images belonging to 96 classes.


In [64]:
model = models.Sequential()
model.add(layers.Conv2D(filters=32, 
                        kernel_size=(2,2),
                        activation='relu',
                        padding = 'same',
                        input_shape=(IMAGE_DIMENSION, IMAGE_DIMENSION, 3),
                        data_format = 'channels_last'))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(16, (3,3), activation='relu'))
model.add(layers.MaxPooling2D((3,3)))
model.add(layers.BatchNormalizationtchNormalization())
model.add(layers.Flatten())
model.add(layers.Dense(64))
model.add(layers.Dense(NUM_CLASS)) # output layer
model.add(layers.Activation('sigmoid'))

model.summary()

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# training begins
model.fit(train_generator, steps_per_epoch=11331 // BATCH_SIZE, epochs=20,
          validation_data=val_generator, callbacks=lr_callback, use_multiprocessing=False)

Model: "sequential_14"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_26 (Conv2D)           (None, 192, 192, 32)      416       
_________________________________________________________________
max_pooling2d_26 (MaxPooling (None, 96, 96, 32)        0         
_________________________________________________________________
conv2d_27 (Conv2D)           (None, 94, 94, 16)        4624      
_________________________________________________________________
max_pooling2d_27 (MaxPooling (None, 31, 31, 16)        0         
_________________________________________________________________
batch_normalization (BatchNo (None, 31, 31, 16)        64        
_________________________________________________________________
flatten_14 (Flatten)         (None, 15376)             0         
_________________________________________________________________
dense_22 (Dense)             (None, 64)              

### simple 11. revert to simple 7. more epochs and smaller batch size

In [65]:
def scheduler(epoch, lr):
    if epoch < 10:
        return lr
    else:
        return 0.00001
    
lr_callback = LearningRateScheduler(scheduler)

In [66]:
BATCH_SIZE = 32
TRAIN_BATCH_SIZE = BATCH_SIZE
VAL_BATCH_SIZE = BATCH_SIZE
TEST_BATCH_SIZE = BATCH_SIZE

In [67]:
train_generator = ImageDataGenerator(rescale=1./255, 
                                     horizontal_flip=True, 
                                     rotation_range=45,
                                     vertical_flip=False,
                                     brightness_range=[0.75,1.25],
                                     zoom_range=0.2,
                                     shear_range=0.2
                                    ).flow_from_directory(
    train_dir,
    target_size=(IMAGE_DIMENSION, IMAGE_DIMENSION), 
    batch_size=TRAIN_BATCH_SIZE,
    shuffle=True 
)

val_generator = ImageDataGenerator().flow_from_directory(
    val_dir,
    target_size=(IMAGE_DIMENSION, IMAGE_DIMENSION),
    batch_size=VAL_BATCH_SIZE,
    shuffle=True
)

test_generator = ImageDataGenerator().flow_from_directory(
    test_dir,
    target_size=(IMAGE_DIMENSION, IMAGE_DIMENSION),
    batch_size=TEST_BATCH_SIZE,
    shuffle=False
)

Found 11331 images belonging to 96 classes.
Found 3666 images belonging to 96 classes.
Found 1271 images belonging to 96 classes.


In [68]:
model = models.Sequential()
model.add(layers.Conv2D(filters=32, 
                        kernel_size=(2,2),
                        activation='relu',
                        padding = 'same',
                        input_shape=(IMAGE_DIMENSION, IMAGE_DIMENSION, 3),
                        data_format = 'channels_last'))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(16, (3,3), activation='relu'))
model.add(layers.MaxPooling2D((3,3)))
model.add(layers.Flatten())
model.add(layers.Dense(64))
model.add(layers.Dense(NUM_CLASS)) # output layer
model.add(layers.Activation('sigmoid'))

model.summary()

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# training begins
model.fit(train_generator, steps_per_epoch=11331 // BATCH_SIZE, epochs=50,
          validation_data=val_generator, callbacks=lr_callback, use_multiprocessing=False)

Model: "sequential_15"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_28 (Conv2D)           (None, 192, 192, 32)      416       
_________________________________________________________________
max_pooling2d_28 (MaxPooling (None, 96, 96, 32)        0         
_________________________________________________________________
conv2d_29 (Conv2D)           (None, 94, 94, 16)        4624      
_________________________________________________________________
max_pooling2d_29 (MaxPooling (None, 31, 31, 16)        0         
_________________________________________________________________
flatten_15 (Flatten)         (None, 15376)             0         
_________________________________________________________________
dense_24 (Dense)             (None, 64)                984128    
_________________________________________________________________
dense_25 (Dense)             (None, 96)              

Epoch 34/50
354/354 [==============================] - 70s 196ms/step - loss: 2.0361 - accuracy: 0.4534 - val_loss: 385.2259 - val_accuracy: 0.2848
Epoch 35/50
354/354 [==============================] - 70s 196ms/step - loss: 2.0221 - accuracy: 0.4646 - val_loss: 385.4923 - val_accuracy: 0.2848
Epoch 36/50
354/354 [==============================] - 69s 196ms/step - loss: 2.0079 - accuracy: 0.4635 - val_loss: 390.3752 - val_accuracy: 0.2821
Epoch 37/50
354/354 [==============================] - 70s 196ms/step - loss: 2.0259 - accuracy: 0.4658 - val_loss: 387.6031 - val_accuracy: 0.2851
Epoch 38/50
354/354 [==============================] - 69s 196ms/step - loss: 2.0239 - accuracy: 0.4655 - val_loss: 388.3347 - val_accuracy: 0.2821
Epoch 39/50
354/354 [==============================] - 69s 195ms/step - loss: 2.0261 - accuracy: 0.4611 - val_loss: 387.2889 - val_accuracy: 0.2845
Epoch 40/50
354/354 [==============================] - 69s 196ms/step - loss: 1.9984 - accuracy: 0.4620 - val_lo

### simple 12. revert to simple 7. use sgd instead of adam

In [69]:
def scheduler(epoch, lr):
    if epoch < 10:
        return lr
    else:
        return 0.00001
    
lr_callback = LearningRateScheduler(scheduler)

In [70]:
BATCH_SIZE = 64
TRAIN_BATCH_SIZE = BATCH_SIZE
VAL_BATCH_SIZE = BATCH_SIZE
TEST_BATCH_SIZE = BATCH_SIZE

In [71]:
train_generator = ImageDataGenerator(rescale=1./255, 
                                     horizontal_flip=True, 
                                     rotation_range=45,
                                     vertical_flip=False,
                                     brightness_range=[0.75,1.25],
                                     zoom_range=0.2,
                                     shear_range=0.2
                                    ).flow_from_directory(
    train_dir,
    target_size=(IMAGE_DIMENSION, IMAGE_DIMENSION), 
    batch_size=TRAIN_BATCH_SIZE,
    shuffle=True 
)

val_generator = ImageDataGenerator().flow_from_directory(
    val_dir,
    target_size=(IMAGE_DIMENSION, IMAGE_DIMENSION),
    batch_size=VAL_BATCH_SIZE,
    shuffle=True
)

test_generator = ImageDataGenerator().flow_from_directory(
    test_dir,
    target_size=(IMAGE_DIMENSION, IMAGE_DIMENSION),
    batch_size=TEST_BATCH_SIZE,
    shuffle=False
)

Found 11331 images belonging to 96 classes.
Found 3666 images belonging to 96 classes.
Found 1271 images belonging to 96 classes.


In [72]:
model = models.Sequential()
model.add(layers.Conv2D(filters=32, 
                        kernel_size=(2,2),
                        activation='relu',
                        padding = 'same',
                        input_shape=(IMAGE_DIMENSION, IMAGE_DIMENSION, 3),
                        data_format = 'channels_last'))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(16, (3,3), activation='relu'))
model.add(layers.MaxPooling2D((3,3)))
model.add(layers.Flatten())
model.add(layers.Dense(64))
model.add(layers.Dense(NUM_CLASS)) # output layer
model.add(layers.Activation('sigmoid'))

model.summary()

model.compile(optimizer='sgd',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# training begins
model.fit(train_generator, steps_per_epoch=11331 // BATCH_SIZE, epochs=20,
          validation_data=val_generator, callbacks=lr_callback, use_multiprocessing=False)

Model: "sequential_16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_30 (Conv2D)           (None, 192, 192, 32)      416       
_________________________________________________________________
max_pooling2d_30 (MaxPooling (None, 96, 96, 32)        0         
_________________________________________________________________
conv2d_31 (Conv2D)           (None, 94, 94, 16)        4624      
_________________________________________________________________
max_pooling2d_31 (MaxPooling (None, 31, 31, 16)        0         
_________________________________________________________________
flatten_16 (Flatten)         (None, 15376)             0         
_________________________________________________________________
dense_26 (Dense)             (None, 64)                984128    
_________________________________________________________________
dense_27 (Dense)             (None, 96)              

### simple 13. revert to simple 7. still need to reduce overfitting. attempt different data augmentations

In [73]:
def scheduler(epoch, lr):
    if epoch < 10:
        return lr
    else:
        return 0.00001
    
lr_callback = LearningRateScheduler(scheduler)

In [74]:
BATCH_SIZE = 64
TRAIN_BATCH_SIZE = BATCH_SIZE
VAL_BATCH_SIZE = BATCH_SIZE
TEST_BATCH_SIZE = BATCH_SIZE

In [75]:
train_generator = ImageDataGenerator(rescale=1./255, 
                                     horizontal_flip=True, 
                                     rotation_range=60,
                                     vertical_flip=False,
                                     brightness_range=[0.4,1.25],
                                     zoom_range=0.2,
                                     shear_range=0.2
                                    ).flow_from_directory(
    train_dir,
    target_size=(IMAGE_DIMENSION, IMAGE_DIMENSION), 
    batch_size=TRAIN_BATCH_SIZE,
    shuffle=True 
)

val_generator = ImageDataGenerator().flow_from_directory(
    val_dir,
    target_size=(IMAGE_DIMENSION, IMAGE_DIMENSION),
    batch_size=VAL_BATCH_SIZE,
    shuffle=True
)

test_generator = ImageDataGenerator().flow_from_directory(
    test_dir,
    target_size=(IMAGE_DIMENSION, IMAGE_DIMENSION),
    batch_size=TEST_BATCH_SIZE,
    shuffle=False
)

Found 11331 images belonging to 96 classes.
Found 3666 images belonging to 96 classes.
Found 1271 images belonging to 96 classes.


In [76]:
model = models.Sequential()
model.add(layers.Conv2D(filters=32, 
                        kernel_size=(2,2),
                        activation='relu',
                        padding = 'same',
                        input_shape=(IMAGE_DIMENSION, IMAGE_DIMENSION, 3),
                        data_format = 'channels_last'))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(16, (3,3), activation='relu'))
model.add(layers.MaxPooling2D((3,3)))
model.add(layers.Flatten())
model.add(layers.Dense(64))
model.add(layers.Dense(NUM_CLASS)) # output layer
model.add(layers.Activation('sigmoid'))

model.summary()

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# training begins
model.fit(train_generator, steps_per_epoch=11331 // BATCH_SIZE, epochs=20,
          validation_data=val_generator, callbacks=lr_callback, use_multiprocessing=False)

Model: "sequential_17"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_32 (Conv2D)           (None, 192, 192, 32)      416       
_________________________________________________________________
max_pooling2d_32 (MaxPooling (None, 96, 96, 32)        0         
_________________________________________________________________
conv2d_33 (Conv2D)           (None, 94, 94, 16)        4624      
_________________________________________________________________
max_pooling2d_33 (MaxPooling (None, 31, 31, 16)        0         
_________________________________________________________________
flatten_17 (Flatten)         (None, 15376)             0         
_________________________________________________________________
dense_28 (Dense)             (None, 64)                984128    
_________________________________________________________________
dense_29 (Dense)             (None, 96)              

### simple 14. revert to simple 7. adding more layers. sgd

In [77]:
def scheduler(epoch, lr):
    if epoch < 10:
        return lr
    else:
        return 0.00001
    
lr_callback = LearningRateScheduler(scheduler)

In [78]:
BATCH_SIZE = 64
TRAIN_BATCH_SIZE = BATCH_SIZE
VAL_BATCH_SIZE = BATCH_SIZE
TEST_BATCH_SIZE = BATCH_SIZE

In [79]:
train_generator = ImageDataGenerator(rescale=1./255, 
                                     horizontal_flip=True, 
                                     rotation_range=45,
                                     vertical_flip=False,
                                     brightness_range=[0.75,1.25],
                                     zoom_range=0.2,
                                     shear_range=0.2
                                    ).flow_from_directory(
    train_dir,
    target_size=(IMAGE_DIMENSION, IMAGE_DIMENSION), 
    batch_size=TRAIN_BATCH_SIZE,
    shuffle=True 
)

val_generator = ImageDataGenerator().flow_from_directory(
    val_dir,
    target_size=(IMAGE_DIMENSION, IMAGE_DIMENSION),
    batch_size=VAL_BATCH_SIZE,
    shuffle=True
)

test_generator = ImageDataGenerator().flow_from_directory(
    test_dir,
    target_size=(IMAGE_DIMENSION, IMAGE_DIMENSION),
    batch_size=TEST_BATCH_SIZE,
    shuffle=False
)

Found 11331 images belonging to 96 classes.
Found 3666 images belonging to 96 classes.
Found 1271 images belonging to 96 classes.


In [80]:
model = models.Sequential()
model.add(layers.Conv2D(filters=32, 
                        kernel_size=(2,2),
                        activation='relu',
                        padding = 'same',
                        input_shape=(IMAGE_DIMENSION, IMAGE_DIMENSION, 3),
                        data_format = 'channels_last'))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(16, (3,3), activation='relu'))
model.add(layers.MaxPooling2D((3,3)))
model.add(layers.Conv2D(8, (4,3), activation='relu'))
model.add(layers.MaxPooling2D((4,3)))
model.add(layers.Flatten())
model.add(layers.Dense(64))
model.add(layers.Dense(NUM_CLASS)) # output layer
model.add(layers.Activation('sigmoid'))

model.summary()

model.compile(optimizer='sgd',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# training begins
model.fit(train_generator, steps_per_epoch=11331 // BATCH_SIZE, epochs=20,
          validation_data=val_generator, callbacks=lr_callback, use_multiprocessing=False)

Model: "sequential_18"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_34 (Conv2D)           (None, 192, 192, 32)      416       
_________________________________________________________________
max_pooling2d_34 (MaxPooling (None, 96, 96, 32)        0         
_________________________________________________________________
conv2d_35 (Conv2D)           (None, 94, 94, 16)        4624      
_________________________________________________________________
max_pooling2d_35 (MaxPooling (None, 31, 31, 16)        0         
_________________________________________________________________
conv2d_36 (Conv2D)           (None, 28, 29, 8)         1544      
_________________________________________________________________
max_pooling2d_36 (MaxPooling (None, 7, 9, 8)           0         
_________________________________________________________________
flatten_18 (Flatten)         (None, 504)             

### simple 15. use simple 14 but with adam.

In [81]:
def scheduler(epoch, lr):
    if epoch < 10:
        return lr
    else:
        return 0.00001
    
lr_callback = LearningRateScheduler(scheduler)

In [82]:
BATCH_SIZE = 64
TRAIN_BATCH_SIZE = BATCH_SIZE
VAL_BATCH_SIZE = BATCH_SIZE
TEST_BATCH_SIZE = BATCH_SIZE

In [83]:
train_generator = ImageDataGenerator(rescale=1./255, 
                                     horizontal_flip=True, 
                                     rotation_range=45,
                                     vertical_flip=False,
                                     brightness_range=[0.75,1.25],
                                     zoom_range=0.2,
                                     shear_range=0.2
                                    ).flow_from_directory(
    train_dir,
    target_size=(IMAGE_DIMENSION, IMAGE_DIMENSION), 
    batch_size=TRAIN_BATCH_SIZE,
    shuffle=True 
)

val_generator = ImageDataGenerator().flow_from_directory(
    val_dir,
    target_size=(IMAGE_DIMENSION, IMAGE_DIMENSION),
    batch_size=VAL_BATCH_SIZE,
    shuffle=True
)

test_generator = ImageDataGenerator().flow_from_directory(
    test_dir,
    target_size=(IMAGE_DIMENSION, IMAGE_DIMENSION),
    batch_size=TEST_BATCH_SIZE,
    shuffle=False
)

Found 11331 images belonging to 96 classes.
Found 3666 images belonging to 96 classes.
Found 1271 images belonging to 96 classes.


In [84]:
model = models.Sequential()
model.add(layers.Conv2D(filters=32, 
                        kernel_size=(2,2),
                        activation='relu',
                        padding = 'same',
                        input_shape=(IMAGE_DIMENSION, IMAGE_DIMENSION, 3),
                        data_format = 'channels_last'))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(16, (3,3), activation='relu'))
model.add(layers.MaxPooling2D((3,3)))
model.add(layers.Conv2D(8, (4,3), activation='relu'))
model.add(layers.MaxPooling2D((4,3)))
model.add(layers.Flatten())
model.add(layers.Dense(64))
model.add(layers.Dense(NUM_CLASS)) # output layer
model.add(layers.Activation('sigmoid'))

model.summary()

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# training begins
model.fit(train_generator, steps_per_epoch=11331 // BATCH_SIZE, epochs=20,
          validation_data=val_generator, callbacks=lr_callback, use_multiprocessing=False)

Model: "sequential_19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_37 (Conv2D)           (None, 192, 192, 32)      416       
_________________________________________________________________
max_pooling2d_37 (MaxPooling (None, 96, 96, 32)        0         
_________________________________________________________________
conv2d_38 (Conv2D)           (None, 94, 94, 16)        4624      
_________________________________________________________________
max_pooling2d_38 (MaxPooling (None, 31, 31, 16)        0         
_________________________________________________________________
conv2d_39 (Conv2D)           (None, 28, 29, 8)         1544      
_________________________________________________________________
max_pooling2d_39 (MaxPooling (None, 7, 9, 8)           0         
_________________________________________________________________
flatten_19 (Flatten)         (None, 504)             

### exact copy of simple 7. running for reproduction

In [85]:
def scheduler(epoch, lr):
    if epoch < 10:
        return lr
    else:
        return 0.00001
    
lr_callback = LearningRateScheduler(scheduler)

In [86]:
BATCH_SIZE = 64
TRAIN_BATCH_SIZE = BATCH_SIZE
VAL_BATCH_SIZE = BATCH_SIZE
TEST_BATCH_SIZE = BATCH_SIZE

In [87]:
train_generator = ImageDataGenerator(rescale=1./255, 
                                     horizontal_flip=True, 
                                     rotation_range=45,
                                     vertical_flip=False,
                                     brightness_range=[0.75,1.25],
                                     zoom_range=0.2,
                                     shear_range=0.2
                                    ).flow_from_directory(
    train_dir,
    target_size=(IMAGE_DIMENSION, IMAGE_DIMENSION), 
    batch_size=TRAIN_BATCH_SIZE,
    shuffle=True 
)

val_generator = ImageDataGenerator().flow_from_directory(
    val_dir,
    target_size=(IMAGE_DIMENSION, IMAGE_DIMENSION),
    batch_size=VAL_BATCH_SIZE,
    shuffle=True
)

test_generator = ImageDataGenerator().flow_from_directory(
    test_dir,
    target_size=(IMAGE_DIMENSION, IMAGE_DIMENSION),
    batch_size=TEST_BATCH_SIZE,
    shuffle=False
)

Found 11331 images belonging to 96 classes.
Found 3666 images belonging to 96 classes.
Found 1271 images belonging to 96 classes.


In [88]:
model = models.Sequential()
model.add(layers.Conv2D(filters=32, 
                        kernel_size=(2,2),
                        activation='relu',
                        padding = 'same',
                        input_shape=(IMAGE_DIMENSION, IMAGE_DIMENSION, 3),
                        data_format = 'channels_last'))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(16, (3,3), activation='relu'))
model.add(layers.MaxPooling2D((3,3)))
model.add(layers.Flatten())
model.add(layers.Dense(64))
model.add(layers.Dense(NUM_CLASS)) # output layer
model.add(layers.Activation('sigmoid'))

model.summary()

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# training begins
model.fit(train_generator, steps_per_epoch=11331 // BATCH_SIZE, epochs=20,
          validation_data=val_generator, callbacks=lr_callback, use_multiprocessing=False)

Model: "sequential_20"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_40 (Conv2D)           (None, 192, 192, 32)      416       
_________________________________________________________________
max_pooling2d_40 (MaxPooling (None, 96, 96, 32)        0         
_________________________________________________________________
conv2d_41 (Conv2D)           (None, 94, 94, 16)        4624      
_________________________________________________________________
max_pooling2d_41 (MaxPooling (None, 31, 31, 16)        0         
_________________________________________________________________
flatten_20 (Flatten)         (None, 15376)             0         
_________________________________________________________________
dense_34 (Dense)             (None, 64)                984128    
_________________________________________________________________
dense_35 (Dense)             (None, 96)              

### adding more epochs

In [89]:
def scheduler(epoch, lr):
    if epoch < 10:
        return lr
    else:
        return 0.00001
    
lr_callback = LearningRateScheduler(scheduler)

In [90]:
BATCH_SIZE = 64
TRAIN_BATCH_SIZE = BATCH_SIZE
VAL_BATCH_SIZE = BATCH_SIZE
TEST_BATCH_SIZE = BATCH_SIZE

In [91]:
train_generator = ImageDataGenerator(rescale=1./255, 
                                     horizontal_flip=True, 
                                     rotation_range=45,
                                     vertical_flip=False,
                                     brightness_range=[0.75,1.25],
                                     zoom_range=0.2,
                                     shear_range=0.2
                                    ).flow_from_directory(
    train_dir,
    target_size=(IMAGE_DIMENSION, IMAGE_DIMENSION), 
    batch_size=TRAIN_BATCH_SIZE,
    shuffle=True 
)

val_generator = ImageDataGenerator().flow_from_directory(
    val_dir,
    target_size=(IMAGE_DIMENSION, IMAGE_DIMENSION),
    batch_size=VAL_BATCH_SIZE,
    shuffle=True
)

test_generator = ImageDataGenerator().flow_from_directory(
    test_dir,
    target_size=(IMAGE_DIMENSION, IMAGE_DIMENSION),
    batch_size=TEST_BATCH_SIZE,
    shuffle=False
)

Found 11331 images belonging to 96 classes.
Found 3666 images belonging to 96 classes.
Found 1271 images belonging to 96 classes.


In [92]:
model = models.Sequential()
model.add(layers.Conv2D(filters=32, 
                        kernel_size=(2,2),
                        activation='relu',
                        padding = 'same',
                        input_shape=(IMAGE_DIMENSION, IMAGE_DIMENSION, 3),
                        data_format = 'channels_last'))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(16, (3,3), activation='relu'))
model.add(layers.MaxPooling2D((3,3)))
model.add(layers.Flatten())
model.add(layers.Dense(64))
model.add(layers.Dense(NUM_CLASS)) # output layer
model.add(layers.Activation('sigmoid'))

model.summary()

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# training begins
model.fit(train_generator, steps_per_epoch=11331 // BATCH_SIZE, epochs=100,
          validation_data=val_generator, callbacks=lr_callback, use_multiprocessing=False)

Model: "sequential_21"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_42 (Conv2D)           (None, 192, 192, 32)      416       
_________________________________________________________________
max_pooling2d_42 (MaxPooling (None, 96, 96, 32)        0         
_________________________________________________________________
conv2d_43 (Conv2D)           (None, 94, 94, 16)        4624      
_________________________________________________________________
max_pooling2d_43 (MaxPooling (None, 31, 31, 16)        0         
_________________________________________________________________
flatten_21 (Flatten)         (None, 15376)             0         
_________________________________________________________________
dense_36 (Dense)             (None, 64)                984128    
_________________________________________________________________
dense_37 (Dense)             (None, 96)              

177/177 [==============================] - 70s 391ms/step - loss: 2.0686 - accuracy: 0.4510 - val_loss: 562.7220 - val_accuracy: 0.2362
Epoch 34/100
177/177 [==============================] - 70s 392ms/step - loss: 2.0957 - accuracy: 0.4497 - val_loss: 556.9500 - val_accuracy: 0.2373
Epoch 35/100
177/177 [==============================] - 70s 392ms/step - loss: 2.0822 - accuracy: 0.4512 - val_loss: 564.5050 - val_accuracy: 0.2340
Epoch 36/100
177/177 [==============================] - 71s 399ms/step - loss: 2.0842 - accuracy: 0.4446 - val_loss: 556.7069 - val_accuracy: 0.2365
Epoch 37/100
177/177 [==============================] - 72s 404ms/step - loss: 2.0856 - accuracy: 0.4424 - val_loss: 571.2230 - val_accuracy: 0.2327
Epoch 38/100
177/177 [==============================] - 71s 401ms/step - loss: 2.0573 - accuracy: 0.4525 - val_loss: 553.0322 - val_accuracy: 0.2390
Epoch 39/100
177/177 [==============================] - 72s 404ms/step - loss: 2.0654 - accuracy: 0.4579 - val_loss: 55

177/177 [==============================] - 69s 389ms/step - loss: 2.0222 - accuracy: 0.4709 - val_loss: 589.0053 - val_accuracy: 0.2300
Epoch 89/100
177/177 [==============================] - 69s 390ms/step - loss: 2.0017 - accuracy: 0.4658 - val_loss: 597.3110 - val_accuracy: 0.2267
Epoch 90/100
177/177 [==============================] - 69s 389ms/step - loss: 2.0177 - accuracy: 0.4596 - val_loss: 600.6490 - val_accuracy: 0.2272
Epoch 91/100
177/177 [==============================] - 69s 390ms/step - loss: 2.0494 - accuracy: 0.4597 - val_loss: 594.1898 - val_accuracy: 0.2291
Epoch 92/100
177/177 [==============================] - 69s 390ms/step - loss: 2.0359 - accuracy: 0.4642 - val_loss: 598.7492 - val_accuracy: 0.2259
Epoch 93/100
177/177 [==============================] - 69s 390ms/step - loss: 2.0092 - accuracy: 0.4637 - val_loss: 597.6760 - val_accuracy: 0.2280
Epoch 94/100
177/177 [==============================] - 69s 389ms/step - loss: 2.0134 - accuracy: 0.4615 - val_loss: 59